In [ ]:
%pip install adlfs==2022.11.2

In [4]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential
import sys

# Enter details of your AML workspace
subscription_id = "89da9f33-fd31-4ece-861e-5fab7af4dc11"
resource_group = "mtcs-dev-aml-rg"
workspace = "mtcs-dev-aml"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

sys.path.append("../") # go to parent dir

In [ ]:
from src.ray_on_aml.core import Ray_On_AML
import logging

ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ray-on-aml-workers")
#Note that if you need to customize the pip installation of the cluster, you also needs to support the ray package e.g. ray[data] which 
#match the version of the ray package(s) in your compute instance. If you don't specify pip_packages then ray[default] is inserted 
#automatically

# For use as client mode, uncomment these lines
ray = ray_on_aml.getRay(num_node=2,pip_packages=["ray[air]==2.2.0","ray[data]==2.2.0","torch==1.13.0","fastparquet==2022.12.0", 
"azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.12.0", "adlfs==2022.11.2", "fsspec==2022.11.0"])
client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")

#use CI as head node

# ray = ray_on_aml.getRay(ci_is_head=True, num_node=2)

In [ ]:
#Ray also support Ray dataset. You can read into ray dataset then convert to Dask or other ML format which is convenient for ML training.https://docs.ray.io/en/latest/data/dataset.html
# you may need to upgrade pyarrow to run this successfully
from adlfs import AzureBlobFileSystem

abfs = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")
#if read all years and months
# data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather//", filesystem=abfs)
data =ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2015/"], filesystem=abfs)
data.count()
# 1,584,481,119 is the count for all data 

In [ ]:
ray_on_aml.shutdown()
# ray.shutdown(True)

In [5]:
from azure.ai.ml import command, Input, Output
from src.ray_on_aml.core import Ray_On_AML
import logging

ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ray-on-aml-workers")

inputs={
    "diabetes": Input(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/data",
    )
}

outputs={
    "output1": Output(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/ray_outputs",
    ),
    "output2": Output(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/ray_outputs",
    )
}


ray = ray_on_aml.getRay(inputs = inputs, outputs=outputs, num_node=2,
pip_packages=["ray[air]==2.2.0","ray[data]==2.2.0","torch==1.13.0","fastparquet==2022.12.0", 
"azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.12.0", "adlfs==2022.11.2", "fsspec==2022.11.0"])
client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")

(raylet, ip=10.0.0.8) /azureml-envs/azureml_c0eb32c1b90d5f184f6500831f6bfab1/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.8)   aiogrpc.init_grpc_aio()


Waiting cluster to start and return head node's ip
.............
 cluster is ready, head node ip  10.0.0.6


In [6]:
data = ray.data.read_csv(ray_on_aml.mount_points['diabetes'])

# data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs)
print(data.count())

2022-12-15 07:00:44,698	WARNING read_api.py:326 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


442


In [ ]:
ray.cluster_resources()
